In [ ]:
%load_ext autoreload

In [ ]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from sandbox import rrtiris

In [ ]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat
from pydrake.all import GenerateSeedingPolytope
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower, FindEpsilonLowerVector, FindEpsilonUpperVector
from pydrake.solvers import mathematicalprogram as mp

# Build Plant


In [ ]:
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]


In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

In [ ]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)


In [ ]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

In [ ]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

In [ ]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

# Setup IRIS Options and Generate Regions

In [ ]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
# # More interesting seed points
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [ 1.3,  2.0, -1.7],  # start: right over left
#                         [ 0.9, -1.3, -1.0],  # goal: left over right
#                         [-0.3, -0.5, -1.7]   # passing
#                          ])
seed_points_q = np.array([[0.0, 0, 0], # zero config
                        [0.8, 1.3, -0.8],  # START: blue low green up
                        [0.1, 0.9, -1.2],     # GOAL: green low other up
                        [0.2, 1.6, -0.6],
                        [0.5, -1.9, -0.9]
                         ])[:,(0,1,2)]   # passing


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)

start = seed_points[1,:]
goal = seed_points[2,:]

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()
q_star = np.zeros(3)

In [ ]:
# regions = []

# iris_options = IrisOptionsRationalSpace()
# iris_options.require_sample_point_is_contained = True
# iris_options.iteration_limit = 20
# iris_options.configuration_space_margin = 1e-5
# iris_options.max_faces_per_collision_pair = 60
# iris_options.termination_threshold = -1
# iris_options.q_star = np.zeros(3)
# iris_options.relative_termination_threshold = 0.05
# iris_options.enable_ibex = False
# #deprecated
# iris_options.certify_region_with_sos_during_generation = False
# iris_options.certify_region_with_sos_after_generation = False

# for i, s in enumerate(seed_points):
#     plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
#     if False:
#         #starting_hpolyhedron = regions[i-1]
#         r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
#                                               iris_options, starting_hpolyhedron)
#     else:
#         r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
#     regions.append(r)
#     print(f'Completed region: {i+1}/{len(seed_points)}')
#     print(f"Sample point contained = {np.all(r.A()@s <= r.b())}")
#     print(f"Sample point contained componentwise = {r.A()@s <= r.b()}")
#     print()

In [ ]:
# if do_viz:
#     visualizer.plot_regions(regions, ellipses=None, region_suffix='original')

In [ ]:
regions_default = []
max_vects = []
min_vects = []
num_perm_dim = 2
num_rot = 2

for i, s in enumerate(seed_points):
    regions_default.append(GenerateSeedingPolytope(s, num_perm_dim, num_rot))
    eps_min_vect = FindEpsilonLowerVector(regions_default[i].A(), 
                                      regions_default[i].b(), 
                                      limits_t[0], limits_t[1], s)
    min_vects.append(eps_min_vect)
    eps_max_vect = FindEpsilonUpperVector(regions_default[i].A(), 
                                      regions_default[i].b(), 
                                      limits_t[0], limits_t[1])
    max_vects.append(eps_max_vect)
    
    

In [ ]:
regions_default_small = []
regions_default_big = []
for i, r in enumerate(regions_default):
    regions_default_small.append(HPolyhedron(regions_default[i].A(), regions_default[i].b() + 0.75*min_vects[i]))
    regions_default_big.append(HPolyhedron(regions_default[i].A(), regions_default[i].b() + max_vects[i]))
    
if do_viz:
    visualizer.plot_regions(regions_default, ellipses=None, region_suffix='_default')
    visualizer.plot_regions(regions_default_small, ellipses=None, region_suffix='_small')
    visualizer.plot_regions(regions_default_big, ellipses=None, region_suffix='_big')

In [ ]:
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)


In [ ]:
i = 3
r0 = regions_default[i]
s = seed_points[i]
filtered_collision_pairs = set()

vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
vector_bisection_search_options.max_iters = 40
vector_bisection_search_options.max_feasible_iters = 5
vector_bisection_search_options.search_d = True
vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(r0.A(), r0.b(),
                                                         limits_t[0], limits_t[1], s)

vector_bisection_search_options.epsilon_max = 0.25*FindEpsilonUpperVector(r0.A(), 
                                      r0.b(), 
                                      limits_t[0], limits_t[1],)
solver_options = mp.SolverOptions()
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

b_feasible = cspace_free_region.CspacePolytopeBisectionSearchVector(
                 q_star, filtered_collision_pairs, r0.A(), r0.b(),
                 vector_bisection_search_options, solver_options)
r0_grown_vector = HPolyhedron(r0.A(), b_feasible) 

In [ ]:

scalar_edited_regions = []
vector_edited_regions = []
filtered_collision_pairs = set()

bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 20
bilinear_alternation_option.lagrangian_backoff_scale = 1e-6
bilinear_alternation_option.polytope_backoff_scale = 1e-6

iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-5
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(3)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
#deprecated
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False

solver_options = mp.SolverOptions()
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

def iris_scalar_handle(seed, domain):
    seed_q = Ratfk.ComputeQValue(seed, np.zeros((3,))) 
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_q)
    region = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, domain)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    C = ellipse.A()
    radii2, R  =  np.linalg.eig(C.T @ C)
    radii = np.sqrt(radii2)

    binary_search_options.epsilon_min = FindEpsilonLower(region.A(), region.b(),
                                                         limits_t[0], limits_t[1])
    print("[IRIS +  CERT]: Eps min", binary_search_options.epsilon_min)
    print("[IRIS +  CERT]: Region found, vol = ", ellipse.Volume(), ", halfaxis: ", radii)
    b_feasible = cspace_free_region.CspacePolytopeBinarySearch(
                 iris_options.q_star, filtered_collision_pairs, region.A(), region.b(),
                 binary_search_options, solver_options)
    
#     C_final, d_final, P_final, q_final = \
#         cspace_free_region.CspacePolytopeBilinearAlternation(
#             iris_options.q_star, filtered_collision_pairs, region.A(), b_feasible,
#            bilinear_alternation_option, solver_options)
    #C_final = np.vstack([C_final, P_joint_limits.A()])
    #d_final = np.concatenate([d_final, P_joint_limits.b()])
    #editted_regions.append(HPolyhedron(C_final, d_final))
    return region, HPolyhedron(region.A(), b_feasible) 

def iris_vector_handle(seed, domain):
    seed_q = Ratfk.ComputeQValue(seed, np.zeros((3,))) 
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_q)
    region = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, domain)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    C = ellipse.A()
    radii2, R  =  np.linalg.eig(C.T @ C)
    radii = np.sqrt(radii2)
    vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(region.A(), region.b(),
                                                         limits_t[0], limits_t[1])
    print("[IRIS +  CERT]: Eps min", vector_bisection_search_options.epsilon_min)
    print("[IRIS +  CERT]: Region found, vol = ", ellipse.Volume(), ", halfaxis: ", radii)
    b_feasible = cspace_free_region.CspacePolytopeBisectionSearchVector(
                 iris_options.q_star, filtered_collision_pairs, region.A(), region.b(),
                 vector_bisection_search_options, solver_options)
    
#     C_final, d_final, P_final, q_final = \
#         cspace_free_region.CspacePolytopeBilinearAlternation(
#             iris_options.q_star, filtered_collision_pairs, region.A(), b_feasible,
#            bilinear_alternation_option, solver_options)
    #C_final = np.vstack([C_final, P_joint_limits.A()])
    #d_final = np.concatenate([d_final, P_joint_limits.b()])
    #editted_regions.append(HPolyhedron(C_final, d_final))
    return region, HPolyhedron(region.A(), b_feasible) 

In [ ]:
if do_viz:
    visualizer.plot_regions([r0_grown_vector], ellipses=None, region_suffix='_grown_vector')

In [ ]:
binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 1
binary_search_options.epsilon_min = -.12
binary_search_options.max_iters = 15
binary_search_options.search_d = True
binary_search_options.epsilon_min = FindEpsilonLower(r0.A(), r0.b(),
                                                         limits_t[0], limits_t[1])
b_feasible = cspace_free_region.CspacePolytopeBinarySearch(
                 iris_options.q_star, filtered_collision_pairs, r0.A(), r0.b(),
                 binary_search_options, solver_options)
r0_grown_scalar = HPolyhedron(r0.A(), b_feasible) 


In [ ]:
if do_viz:
#     visualizer.plot_regions([r0_grown_scalar], ellipses=None, region_suffix='_grown_scalar')
    visualizer.plot_regions([r0_grown_vector], ellipses=None, region_suffix='_grown_vector')

In [ ]:
#refine with CPP code:
cpp_editted_vector_regions = []
# for i in range(seed_points.shape[0]):
for i in range(1):
    s = seed_points[i,:]
    region_init = regions[i]
    try:
        region_init, region = iris_vector_handle(s, region_init)
        cpp_editted_vector_regions.append(region)
    except Exception as e:
        print(e)
    print(f"Completed {i+1}/{seed_points.shape[0]}")

In [ ]:
d_final = np.array([0.939411,
     1.35849,
    0.939411,
    0.939411,
     1.35849,
    0.939411,
   0.0526289,
   0.0260675,
    0.420636,
   0.0328898,
  0.00448928,
   0.0308143,
-0.000379762,
    0.180957,
    0.156639,
    0.129082,
    0.121289,
     0.63852])
d_without_epsilon= np.array([1.13833,
 1.55741,
 1.13833,
 1.13833,
 1.55741,
 1.13833,
 0.25155,
0.224989,
0.619558,
0.231811,
0.203411,
0.229736,
0.198542,
0.379879,
 0.35556,
0.328004,
 0.32021,
0.837441])
eps_min = np.array(d_final-d_without_epsilon)
print(eps_min)

In [ ]:
#refine with CPP code:
cpp_editted_scalar_regions = []
for i in range(seed_points.shape[0]):
    s = seed_points[i,:]
    region_init = regions[i]
    region_init, region = iris_scalar_handle(s, region_init)
    cpp_editted_scalar_regions.append(region)
    print(f"Completed {i+1}/{seed_points.shape[0]}")

In [ ]:
if do_viz:
    visualizer.plot_regions(cpp_editted_regions, ellipses=None, region_suffix='cpp_editted_regions')

In [ ]:
#setup refine with python
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': -1
}
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)
regions, ellipses = iris_generator.iris_in_rational_space(seed_points)
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

In [ ]:
if do_viz:
    visualizer.plot_regions(iris_generator.regions, region_suffix = '_python_snopt')

In [ ]:
# refine with python
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

In [ ]:
if do_viz:
    visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')

In [ ]:
from pydrake.all import Solve, eq, le, ge

#lam*(binner_new - pullback) - bouter<= -1e-10
def solve_pullback(b_outer, b_inner_new, lam, tol = 1e-9):
    prog = MathematicalProgram()
    pullback = prog.NewContinuousVariables(len(b_inner_new), 'pullback')
    prog.AddLinearConstraint(-lam, -np.inf*np.ones(len(b_outer)).reshape(-1,1) ,b_outer - tol - lam@b_inner_new, pullback)
    prog.AddBoundingBoxConstraint(0, np.inf, pullback)
    res = Solve(prog)
    pullback_sol = res.GetSolution(pullback)
    return res.is_success(), pullback_sol
    
def solve_hpoly_containment_problem(r_outer, r_inner):
    A_outer = r_outer.A()
    A_inner = r_inner.A()
    b_outer = r_outer.b()
    b_inner = r_inner.b()
    prog = MathematicalProgram()
    lam = prog.NewContinuousVariables(len(b_outer), len(b_inner), 'lam')
    prog.AddBoundingBoxConstraint(0.0, np.inf, lam)
    for col in range(A_outer.T.shape[1]):
        prog.AddLinearEqualityConstraint(A_inner.T, A_outer.T[:,col], lam.T[:,col])
        
    
    for r in range(lam.shape[0]):
        prog.AddLinearConstraint(b_inner[:,np.newaxis].T, np.array([-np.inf]), np.array([b_outer[r]- 1e-10]), lam[r, :])

    res =  Solve(prog)
    lam_sol = res.GetSolution(lam)
    return res.is_success(), lam_sol

def solve_volume_maximization_step(r_outer, r_inner, lam_sol, seed_point):
    A_inner = r_inner.A()
    b_inner = r_inner.b()
    b_outer = r_outer.b()
    dim = A_inner.shape[1]
    print('here')
    prog = MathematicalProgram()
    d = prog.NewContinuousVariables(dim, 'd')
    C = prog.NewSymmetricContinuousVariables(dim, 'C')
    eps = prog.NewContinuousVariables(A_inner.shape[0], 'eps')
    prog.AddMaximizeLogDeterminantCost(C)
    for idx in range(len(b_inner)):
        z = prog.NewContinuousVariables(dim + 1)
        z[0] = b_inner[idx] + eps[idx] - A_inner[idx, :]@d
        z[1:] = C @ A_inner[idx, :].T
        prog.AddLorentzConeConstraint(z)  
    prog.AddLinearConstraint(lam_sol, -np.inf*np.ones_like(b_outer), b_outer - lam_sol@b_inner, eps)
    prog.AddLinearConstraint(np.eye(len(eps)), A_inner@seed_point - b_inner, np.inf*np.ones_like(eps), eps)
    #prog.AddConstraint(le(lam_sol@(b_inner + eps) - b_outer, 0))
    #prog.AddConstraint(le(A_inner@seed_point , b_inner + eps))
    prog.AddBoundingBoxConstraint(0.0, 10.0, eps)
    res = Solve(prog)
    
    eps_sol = res.GetSolution(eps)
    print('here2')
    print(eps_sol)
    new_reg = HPolyhedron(A_inner, b_inner + eps_sol)
    return res.is_success(), eps_sol, res.get_optimal_cost()


def inner_approx_HPoly_bilinear_alternation(r_outer, r_inner_seed, seed_point, maxit = 10):
    #check if contained at start
    if r_inner_seed.ContainedInOtherHPolyhedron(r_outer):
        r_inner = r_inner_seed
        past_regions = []
        for it in range(maxit):
            print('step', it)
            #get certificate of containment
            success, lam_sol = solve_hpoly_containment_problem(r_outer, r_inner)
            print('containment step: ', success, 'lambda min', np.min(lam_sol))
            if not success:
                print('terminated early, containment lost')
                return past_regions[-1]
            #use certificate to push out faces of inner polytope
            success, eps, vol_surrogate = solve_volume_maximization_step(r_outer, r_inner_seed, lam_sol, seed_point)
            print('growth step: ', success, 'volume surrogate: ', vol_surrogate)
            print('eps', eps)
            #update polytope
            #success, pullback = solve_pullback(r_outer.b(), r_inner_seed.b() + eps, lam_sol)
            #|print('pullback: ', success, 'pullback ineq', np.max(lam_sol@(r_inner_seed.b() + eps - pullback)-r_outer.b()))
            
            r_inner = HPolyhedron(r_inner.A(), r_inner_seed.b() + eps - 1e-10) #-pullback
            print(f'max inscribed ellipse volume: {r_inner.MaximumVolumeInscribedEllipsoid().Volume()}')
            past_regions.append(r_inner)
        return r_inner
    else:
        raise ValueError("Seed HPolytope not contained in outer Hpolytope")

In [ ]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-6
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(3)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
def iris_handle(seed, domain):
    seed_q = Ratfk.ComputeQValue(seed, np.zeros((3,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_q)
    region = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, domain)
    return region
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])

In [ ]:
compare_seed = seed_points[4]
iris_reg = iris_handle(compare_seed, starting_poly)
sym_poly = GenerateSeedingPolytope(compare_seed, 2, 2)
eps_min_vect = FindEpsilonLowerVector(sym_poly.A(),
                                      sym_poly.b(),
                                      limits_t[0], limits_t[1], compare_seed)

visualizer.plot_regions([iris_reg], region_suffix = 'bil_iris')
visualizer.plot_regions([sym_poly], region_suffix = 'bil_sym')

In [ ]:
small_sym_poly = HPolyhedron(sym_poly.A(), sym_poly.b() + .99 * eps_min_vect)
visualizer.plot_regions([small_sym_poly], region_suffix = 'bil_small_sym')

In [ ]:
print(eps_min_vect)
print(small_sym_poly.MaximumVolumeInscribedEllipsoid().Volume())
print(small_sym_poly.PointInSet(compare_seed))
print(small_sym_poly.ContainedInOtherHPolyhedron(iris_reg))
print(iris_reg.PointInSet(compare_seed))

In [ ]:
r_new = inner_approx_HPoly_bilinear_alternation(iris_reg, small_sym_poly, compare_seed, maxit = 40)
visualizer.plot_regions([r_new], region_suffix = 'bil_new')
print(r_new.MaximumVolumeInscribedEllipsoid().Volume())
print(r_new.PointInSet(compare_seed))
print(r_new.ContainedInOtherHPolyhedron(iris_reg))

In [ ]:
for r_new in regions_new:
    print(f'max inscribed ellipse volume: {r_new.MaximumVolumeInscribedEllipsoid().Volume()}')
    print(r_new.PointInSet(compare_seed))
    print('contained', r_new.ContainedInOtherHPolyhedron(iris_reg))
visualizer.plot_regions(regions_new, region_suffix = 'bil_regions_new4')


In [ ]:
visualizer.plot_regions([regions_new[-1]], region_suffix = 'end')

In [ ]:
iris_reg.A().shape